<left>FINM 33160 - Machine Learning In Finance</left>
<left>Winter 2023</left>
<br>
<h1><center> Homework 2 </center></h1>
<center> Due - 23:59 [CST] January 29th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

<h2> Imports </h2>

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib import cm
from collections import Counter
sns.set()

<h2> Data </h2>

In [4]:
df = pd.read_pickle(r'.\dataset.pkl')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '.\\dataset.pkl'

<h2> Q1 </h2>

- Inserting a column

In [ ]:
df['Q1'] =